# 캠페인 및 상호 작용 보기

첫 번째 노트북 `Personalize_BuildCampaign.ipynb`에서는 Amazon Personalize에 딥 러닝을 사용하여 추천 모델을 구축하고 배포했습니다.

이 노트북에서는 거기서 확장하여 사용자의 실시간 행동에 반응하는 기능을 추가하는 과정을 안내합니다. 영화를 검색하는 동안 사용자의 의도가 바뀌면 해당하는 행동에 따라 수정되는 추천을 확인할 수 있습니다.

또한 추천이 반환되기 전에 영화를 선택하는 사용자 행동을 시뮬레이션하기 위한 데모 코드도 보여줍니다.

아래에서는 Personalize와 상호 작용해야 하는 라이브러리를 가져오는 것부터 시작합니다.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

아래 코드 줄은 첫 번째 노트북에서 공유 변수를 검색합니다.

In [ ]:
%store -r

In [ ]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## 이벤트 트래커 생성

추천 시스템이 실시간 이벤트에 응답하려면, 이벤트 트래커가 필요합니다. 아래의 코드는 이벤트 트래커를 생성합니다. 해당 이벤트 트래커를 이 실습에서 계속 사용할 수 있습니다. 보다 유용한 이름으로 자유롭게 바꾸세요.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

In [ ]:
event_tracker_arn = response['eventTrackerArn']

## 소스 데이터 구성

위에서 추적 ID를 확인할 수 있는데, 이 ID는 변수에 할당되었으므로 추가 작업이 필요하지 않습니다. 아래의 코드 줄은 나중에 영화 목록을 렌더링할 수 있도록 추천에 사용되는 데이터를 설정합니다.

In [ ]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.loc[movie_id]['TITLE']

## 추천 받기

먼저 이전 노트북의 추천을 다시 렌더링하겠습니다.

In [ ]:
recommendations_df

## 사용자 행동 시뮬레이션

아래의 코드 줄은 특정 항목과 상호 작용하는 사용자를 시뮬레이션하는 코드 샘플을 제공하며, 여기서는 시작할 때와 다른 추천을 받게 됩니다.

In [ ]:
session_dict = {}

In [ ]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
)

이 줄 바로 아래에서는 사용자가 특정 타이틀을 누른 것처럼 트래커가 업데이트됩니다.


아래 셀에 의해 생성된 테이블에 의해 추천이 변경되지 않는 경우, 위의 셀에서 다른 임의의 3자리 숫자를 선택하여 두 셀을 다시 실행합니다. 추천이 생성된 세 번째 열이 나타납니다.

In [ ]:
# Pick a movie, we will use ID 270 or Gattaca
movie_to_click = 270
movie_title_clicked = get_movie_title(movie_to_click)
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)

이 블록을 실행하고 나면, 이벤트 추적을 활성화하고 서비스에 이벤트를 전송한 경우 추천이 변경됩니다.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

## 결론

이제 사용자가 상호 작용하는 영화를 변경하면 추천이 변경된다는 것을 알 수 있습니다. 이 시스템은 사용자가 항목 컬렉션과 상호 작용하는 모든 애플리케이션으로 수정할 수 있습니다. 이러한 도구는 언제든지 사용할 수 있으며, 현재 보유하고 있는 데이터를 어떻게 활용할지 살펴볼 수 있습니다.

아래의 셀을 실행하여 정리 노트북에 필요한 값을 저장합니다.

마지막으로 계정에서 항목을 제거할 준비가 되었으면 `Cleanup.ipynb` 노트북을 열고 제거 단계를 실행합니다.


In [ ]:
%store event_tracker_arn